In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers datasets torch scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


# **Imports**

In [3]:

import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from tqdm import tqdm
import pandas as pd
from torch.utils.data import DataLoader
from torch.optim import AdamW
import torch.nn.functional as F

In [4]:
data = pd.read_csv('/content/drive/My Drive/Bachelorprojekt/auswahl(1).csv') #Path to a file where preprocessing already happened

In [5]:
texts = data['no stopwords'].tolist()
labels = data['class'].tolist()

# Split the data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts, labels, test_size=0.2, stratify=labels
)

# Create a DataFrame for validation data
#val_data = pd.DataFrame({
#    'no stopwords': val_texts,  # Texts without stopwords
#    'class': val_labels  # Corresponding labels
#})

# Save the validation DataFrame as a CSV file
#val_data.to_csv('/content/drive/My Drive/Bachelorprojekt/val_data_no_stopwords.csv', index=False)

# Confirmation message
#print("Validation data has been successfully saved to a CSV file.")

In [6]:
train_texts = [str(text) for text in train_texts]  # Convert all training texts to strings
val_texts = [str(text) for text in val_texts]  # Convert all validation texts to strings

In [7]:
tokenizer = AutoTokenizer.from_pretrained('cardiffnlp/twitter-roberta-base-hate')
#tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
#tokenizer = AutoTokenizer.from_pretrained('Hate-speech-CNERG/dehatebert-mono-english')

# Tokenize the training and validation texts
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/700 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [8]:
class TextDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)  # Return the total number of labels

    def __getitem__(self, idx):
        # Create a dictionary for the given index with encoded inputs and corresponding label
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])  # Add the label for the current index
        return item

# Create PyTorch datasets for training and validation
train_dataset = TextDataset(train_encodings, train_labels)
val_dataset = TextDataset(val_encodings, val_labels)

In [9]:
# Assuming there are two classes (Binary Classification)

model = AutoModelForSequenceClassification.from_pretrained('cardiffnlp/twitter-roberta-base-hate', num_labels=2)
#model = AutoModelForSequenceClassification.from_pretrained('Hate-speech-CNERG/dehatebert-mono-english', num_labels=2)
#model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)
optimizer = AdamW(model.parameters(), lr=5e-5)

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [10]:
# Use GPU if available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

for epoch in range(3):  # Number of epochs
    model.train()
    loop = tqdm(train_loader, leave=True)  # Progress bar for the training loop
    for batch in loop:
        # Move batch data to GPU
        batch = {key: val.to(device) for key, val in batch.items()}

        # Forward and backward propagation
        outputs = model(**batch)
        loss = outputs.loss
        optimizer.zero_grad()  # Clear gradients
        loss.backward()  # Backpropagation
        optimizer.step()  # Update model parameters

        # Display progress
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

Epoch 2: 100%|██████████| 150/150 [00:45<00:00,  3.32it/s, loss=0.244]


# **Model Evaluation**

In [11]:
model.eval()
predictions, true_labels = [], []
threshold = 0.5  # Threshold for the "sexist" class

with torch.no_grad():
    for batch in val_loader:
        # Move batch to the device (GPU/CPU)
        batch = {key: val.to(device) for key, val in batch.items()}

        # Get predictions from the model
        outputs = model(**batch)
        logits = outputs.logits

        # Calculate probabilities using Softmax
        probabilities = F.softmax(logits, dim=1)  # [batch_size, num_classes]

        # Apply manual threshold (Class "1" = sexist)
        batch_predictions = (probabilities[:, 1] > threshold).int()  # Class 1 if P > 0.5
        predictions.extend(batch_predictions.cpu().numpy())

        # Collect true labels
        true_labels.extend(batch['labels'].cpu().numpy())

# Calculate accuracy
accuracy = accuracy_score(true_labels, predictions)
print(confusion_matrix(true_labels, predictions))
print(f'Validation Accuracy with Threshold {threshold}: {accuracy}')

[[238  79]
 [ 50 233]]
Validation Accuracy with Threshold 0.5: 0.785


# **Saving Models**

In [12]:
save_directory = "/content/drive/My Drive/Bachelorprojekt/final_model"

# Save the model and tokenizer
model.save_pretrained(save_directory)  # Save the trained model to the specified directory
tokenizer.save_pretrained(save_directory)  # Save the tokenizer to the same directory

('/content/drive/My Drive/Bachelorprojekt/final_model/tokenizer_config.json',
 '/content/drive/My Drive/Bachelorprojekt/final_model/special_tokens_map.json',
 '/content/drive/My Drive/Bachelorprojekt/final_model/vocab.json',
 '/content/drive/My Drive/Bachelorprojekt/final_model/merges.txt',
 '/content/drive/My Drive/Bachelorprojekt/final_model/added_tokens.json',
 '/content/drive/My Drive/Bachelorprojekt/final_model/tokenizer.json')

roberta tweet mit 3 epochen:
[[227  90]
 [ 35 248]]
Validation Accuracy with Threshold 0.5: 0.7916666666666666

roberta base mit 3 epochen:
[[262  55]
 [ 87 196]]
Validation Accuracy with Threshold 0.5: 0.7633333333333333


berthate mit 3 epochen:
[[258  59]
 [ 70 213]]
Validation Accuracy with Threshold 0.6: 0.785






# **Roberta Tweet mit treshold 0.5:**

**1:**
[[227  90]
 [ 35 248]]

Validation Accuracy with Threshold 0.5: 0.7916666666666666

------------------------------------

**2:**
[[270  47]
  [ 66 217]]
  
Validation Accuracy with Threshold 0.5: 0.8116666666666666

---------------------------------------

**3:**
[249  68]
 [ 57 226]]

Validation Accuracy with Threshold 0.5: 0.7916666666666666

----------------------------------------

**4:**
[[251  66]
 [ 62 221]]

Validation Accuracy with Threshold 0.5: 0.7866666666666666


----------------------------------------

**5:**
[[246  71]
 [ 45 238]]

Validation Accuracy with Threshold 0.5: 0.8066666666666666


----------------------------------------

**6:**
[[222  95]
 [ 25 258]]

Validation Accuracy with Threshold 0.5: 0.8


----------------------------------------

**7:**
[[267  50]
 [ 67 216]]

Validation Accuracy with Threshold 0.5: 0.805


----------------------------------------

**8:**
[[254  63]
 [ 79 204]]

Validation Accuracy with Threshold 0.5: 0.7633333333333333



----------------------------------------

**9:**
[[271  46]
 [ 97 186]]

Validation Accuracy with Threshold 0.5: 0.7616666666666667



----------------------------------------

**10:**
[[236  81]
 [ 53 230]]

Validation Accuracy with Threshold 0.5: 0.7766666666666666



----------------------------------------

**11:**
[[276  41]
 [ 80 203]]

Validation Accuracy with Threshold 0.5: 0.7983333333333333